# Collect bookspines
Collect from firestore and storage

In [1]:
#firestore

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import os, json

cred = credentials.Certificate(r"C:\!Dev\projects\book_detection\keys\avea-biblosphere-210106-firebase-adminsdk-s3bjc-ff2c6ac275 (1).json")
firebase_admin.initialize_app(cred)

db = firestore.client()

In [32]:
# storage
from google.cloud import storage

storage_client = storage.Client.from_service_account_json('biblosphere-210106-0e61a72c6976.json')
bucket = storage_client.get_bucket('biblosphere-210106.appspot.com')

In [53]:
#bigquery
from google.oauth2 import service_account
from google.cloud import bigquery

project_id = 'biblosphere-210106'
credentials = service_account.Credentials.from_service_account_file('./keys/biblosphere-210106-46dbcd9058f7.json')

bq_client = bigquery.Client(credentials= credentials,project=project_id)

FileNotFoundError: Could not find module 'C:\ProgramData\Anaconda3\Library\bin\geos_c.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [ ]:
#create table
from google.cloud import bigquery

#client = bigquery.Client()

table_id = "biblosphere-210106.biblosphere.finded_unfinded_books"

schema = [
    bigquery.SchemaField("photo_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("result", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("bookspine", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("book", "JSON", mode="NULLABLE"),
    bigquery.SchemaField("url", "STRING", mode="NULLABLE")
]

table = bigquery.Table(table_id, schema=schema)
table = bq_client.create_table(table)
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

In [ ]:
def get_books_from_bucket(data):
    filename = data['photo']
    json_filename = os.path.splitext(filename)[0] + '.json'
    jsonfile_blob = bucket.blob(json_filename)

    json_data = json.loads(jsonfile_blob.download_as_string())
    recognized = json_data['recognized'] if 'recognized' else []
    unrecognized = json_data['unrecognized'] if 'unrecognized' else []

    return recognized, unrecognized

In [52]:
res = db.collection('photos').where(u'recognition_stats.record_in_stats', u'==', True).stream()

count = 0
for n, doc in enumerate(res):
   count += 1
   if count > 1:
      break

   photo_id = doc.id
   print(f'{n}) {photo_id}')
   data = doc.to_dict()
   url = data['url']
   recognized, unrecognized = get_books_from_bucket(data)

   for book in recognized:
      record = {'photo_id': photo_id,
                'result': 'recognized',
                'bookspine': book['bookspine'],
                'book': book['book'],
                'url': url
                }
      print(record)

   for book in unrecognized:
       record = {'photo_id': photo_id,
                 'result': 'unrecognized',
                 'bookspine': book['bookspine'],
                 'book': None,
                 'url': url
                 }
       print(record)







0) 03E2LrAHsJucumq0el4Q
{'photo_id': '03E2LrAHsJucumq0el4Q', 'result': 'recognized', 'bookspine': 'кровь пот и пиксели десен', 'book': {'_type': 'Book', 'authors': 'Шрейер Джейсон', 'back_text': '', 'cover_text': '', 'description': '', 'genre': '', 'image': 'https://cdn1.ozone.ru/multimedia/c250/1023291146.jpg', 'isbn': '9785040925612', 'keys': ['кровь', 'пот', 'и', 'пиксели', 'обратная', 'сторона', 'индустрии', 'видеоигр', 'шрейер', 'джейсон'], 'language': 'rus', 'tags': '', 'title': 'Кровь, пот и пиксели. Обратная сторона индустрии видеоигр'}, 'url': 'https://firebasestorage.googleapis.com:443/v0/b/biblosphere-210106.appspot.com/o/images%2FPChIJi-dY2A8xlkYRtwyBveXn2tY%2F1625910544469.jpg?alt=media&token=564e5d37-f053-43fc-94ee-4585cc284aaa'}
{'photo_id': '03E2LrAHsJucumq0el4Q', 'result': 'recognized', 'bookspine': 'борзов батл с кадровиком', 'book': {'_type': 'Book', 'authors': 'Борзов Ричард', 'back_text': '', 'cover_text': '', 'description': '', 'genre': '', 'image': 'https://cdn1.